In [2]:
import xarray as xr
import dscim
import yaml
from dscim.menu.simple_storage import Climate, EconVars
import pandas as pd
import numpy as np
from xarray.testing import assert_allclose
from xarray.testing import assert_equal
from itertools import product

import dask
import logging
import subprocess
from subprocess import CalledProcessError
from abc import ABC, abstractmethod
from dscim.descriptors import cachedproperty
from itertools import product
from dscim.menu.decorators import save
from dscim.menu.simple_storage import StackedDamages, EconVars
from dscim.utils.utils import (
    model_outputs,
    compute_damages,
    c_equivalence,
    power,
    quantile_weight_quantilereg,
    extrapolate,
)

In [13]:
# Config vs function params:
# Config should have all paths and parameters that are messy

# function params should be all of the simple parameters that catually want to be changed by EPA

def epa_scc(eta = 2.0,
            rho = 0.0,
            pulse_year = 2020,
            discount_type = "euler_ramsey",
            menu_option = "risk_aversion",
            gases = ['CO2_Fossil', 'CH4', 'N2O'],
            weitzman_parameters = [0.1, 0.5],
            fair_aggregation = ["ce", "mean", "gwr_mean", "median"],
            ssps = ['SSP2', 'SSP3', 'SSP4']):
    
    USER = "liruixue"
    master = f"/home/{USER}/repos/dscim-epa/damage_fun_runs/master-CAMEL_m1_c0.20.yaml"

    
    with open(master, "r") as stream:
        conf = yaml.safe_load(stream)
    
    conf["global_parameters"] = {'fair_aggregation': fair_aggregation,
     'subset_dict': {'ssp': ssps},
     'weitzman_parameter': weitzman_parameters,
     'save_files': ['uncollapsed_sccs']}

    conf['global_parameters']['fair_aggregation'] 
    save_path = "/home/liruixue/replication_newcode/"


    econ = EconVars(
        path_econ="/shares/gcp/integration_replication/inputs/econ/rff_global_socioeconomics.nc4"
    )

    MENU_OPTIONS = {
        "adding_up": dscim.menu.baseline.Baseline,
        "risk_aversion": dscim.menu.risk_aversion.RiskAversionRecipe,
        "equity": dscim.menu.equity.EquityRecipe,
    }
    add_kwargs = {
        "econ_vars": econ,
        "climate_vars": Climate(**conf["rff_climate"], pulse_year=pulse_year),
        "formula": 'damages ~ -1 + anomaly + np.power(anomaly, 2) + gmsl + np.power(gmsl, 2)',
        "discounting_type": discount_type,
        "sector": "CAMEL_m1_c0.20",
        "ce_path": None,
        "save_path": save_path,
        "eta": eta,
        "rho": rho,
        "damage_function_path": "/shares/gcp/integration_replication/results/rff/CAMEL_m1_c0.20/2020",
        "ecs_mask_path": None,
        "ecs_mask_name": None,
    }

    kwargs = conf["global_parameters"].copy()
    for k, v in add_kwargs.items():
        assert (
            k not in kwargs.keys()
        ), f"{k} already set in config. Please check `global_parameters`."
        kwargs.update({k: v})

    menu_item = MENU_OPTIONS[menu_option](**kwargs)
    menu_item.order_plate("scc")
    uncollapsed_scc = menu_item.uncollapsed_sccs
    
    return(menu_item)
    
# This represents the full gamut of scc runs when run default
def epa_sccs(etas_rhos = [[2.0, 0.0],
               [1.016010255, 9.149608e-05],
               [1.244459066, 0.00197263997],
               [1.421158116, 0.00461878399],
               [1.567899395, 0.00770271076]],
             risk_combos = [['risk_aversion', 'euler_ramsey'],
                            ['adding_up','constant']],
             pulse_year = 2020,
             gases = ['CO2_Fossil', 'CH4', 'N2O'],
             weitzman_parameters = [0.1, 0.5],
             fair_aggregation = ["ce", "mean", "gwr_mean", "median"],
             ssps = ['SSP2', 'SSP3', 'SSP4']):

    
    for i, j in product(etas_rhos, risk_combos):
        epa_scc(discount_type = j[1],
                menu_option = j[0],
                eta = i[0],
                rho = i[1],
                pulse_year = 2020,
                gases = gases,
                weitzman_parameters = weitzman_parameters,
                fair_aggregation = fair_aggregation,
                ssps = ssps)    




In [14]:
#etas_rhos = [[2.0, 0.0],
#             [1.016010255, 9.149608e-05],
#             [1.244459066, 0.00197263997],
#             [1.421158116, 0.00461878399],
#             [1.567899395, 0.00770271076]]
etas_rhos = [[2.0,0.0]]
risk_combos = [['risk_aversion', 'euler_ramsey'],
               ['adding_up','constant']]
pulse_year = 2020
gases = ['CO2_Fossil', 'CH4', 'N2O']
weitzman_parameters = [0.1, 0.5]
fair_aggregation = ["ce", "mean", "gwr_mean", "median"]
ssps = ['SSP2', 'SSP3', 'SSP4']
epa_sccs(etas_rhos,
        risk_combos,
        pulse_year=2020)    


 Executing 
        Running risk_aversion
        sector: CAMEL_m1_c0.20
        discounting: euler_ramsey
        eta: 2.0
        rho: 0.0
        
Processing damage functions ...
Skipped saving damage_function_coefficients.
Subsetting on {} keys.
Global consumption found up to 2300.
Skipped saving global_consumption.
Skipped saving damage_function_coefficients.
Skipped saving global_consumption_no_pulse.
Processing SCC calculation ...
Subsetting on {} keys.
Skipped saving ce_fair_no_pulse.
Skipped saving discount_factors.
Skipped saving ce_fair_no_pulse.
Skipped saving damage_function_coefficients.
Skipped saving global_consumption_pulse.
Skipped saving ce_fair_pulse.
Skipped saving marginal_damages.
Skipped saving ce_fair_no_pulse.
Saving /home/liruixue/replication_newcode/risk_aversion_euler_ramsey_eta2.0_rho0.0_uncollapsed_sccs.nc4
Skipped saving scc.
Skipped saving uncollapsed_marginal_damages.
Skipped saving uncollapsed_discount_factors.
Results available: /home/liruixue/repli